In [1]:
import pyspark

En faisant appel à la méthode <code>SparkContext()</code> nous partons une session Spark, créant un objet qui encapsule tout le nécessaire pour ‘communiquer avec’ un cluster Spark. Il est une convention de nommer cet objet  <code>sc</code>. C’est ce que vous trouverez sur des exemples dans la documentation officielle de Spark, ainsi que ailleurs sur le web. 

In [2]:
sc = pyspark.SparkContext()

Maintenant que nous avons une session Spark active dans l’objet <code>sc</code>, nous sommes prêts pour créer des RDDs. Il y a deux façons pour créer un RDD. Ici, nous examinons la prémière: la méthode <code>parallelize</code>

In [3]:
# Let's create an RDD containing a small list with integers for elements:

some_numbers = [1,2,3,4,5,6,7,8,9,10]

my_first_rdd = sc.parallelize(some_numbers)

In [ ]:
my_first_rdd

Qu’est-ce que nous avons fait?

Spark a pris notre liste de chiffres et l’a brisée en plusieurs morceaux. On nomme ces morceaux des **Partitions**. Chacune de ces partitions peut être manipulée de manière indépendante par des Executors, ce qui permet à Spark de ‘diviser pour mieux régner' et de réaliser des opérations de calcul sur vos données en parallèle!

In [6]:
# Regardons en combien de partitions Spark a divisé notre liste
my_first_rdd.

40

In [7]:
# Regardons ce qu'il y a dans nos partitions

my_first_rdd.

[[],
 [],
 [],
 [1],
 [],
 [],
 [],
 [2],
 [],
 [],
 [],
 [3],
 [],
 [],
 [],
 [4],
 [],
 [],
 [],
 [5],
 [],
 [],
 [],
 [6],
 [],
 [],
 [],
 [7],
 [],
 [],
 [],
 [8],
 [],
 [],
 [],
 [9],
 [],
 [],
 [],
 [10]]

Le nombre de partitions est l’un des paramètres importants d’un programme Spark dont vous devez prendre connaissance. Divisez vos données en un nombre trop petit de partitions et Spark ne pourra pas faire autant d’opérations en parallèle que ce que le Hardware dans votre cluster le permettrait. Divisez vos données en un trop grand nombre de partitions et vous aurez des partitions vides, ou bien, vous ne profiterez pas complètement du parallélisme de Spark, car les Executors travailleront sur un grand nombre de petites tâches en séquence.

Ici, nous demandons à Spark d'utiliser 10 partitions:

In [4]:
my_first_rdd_repartitioned = my_first_rdd.
my_first_rdd_repartitioned.getNumPartitions()

10

L’API RDD continent deux types de méthodes: les **Transformations** et les **Actions**. Les Transformations sont des opérations sur des RDDs dont le résultat est un autre RDD. Les Actions sont des opérations sur des RDDs dont le résultat **n’est pas** un autre RDD. Dans la ligne ci-dessus, <code>repartition</code> est une Transformation et <code>getNumPartitions</code> est une Action. Voici quelques exemples pour voir ce concepte en pratique:

In [5]:
# Notre première 'vraie' transformation: additionons +1 à chaque chiffre dans notre liste:

my_first_rdd_repartitioned.

PythonRDD[6] at RDD at PythonRDD.scala:48

La méthode <code>map</code> applique une fonction à chaque élément de chaque partition d’un RDD. Le résultat nous dit que cela a retourné un autre RDD. Comment faire pour examiner le contenu de ce RDD? D’abord, il faut le transférer du Cluster au Driver.

In [13]:
# La méthode collect() transfert le contenu d'un RDD du Cluster au Driver, où nous pouvons le voir:

my_first_rdd_repartitioned.

[7, 4, 8, 9, 1, 10, 5, 2, 6, 3]


Nos chiffres ne sont plus en ordre, mais nous avons toujours notre liste de chiffres de 1 à 10. Nous avions appliqué une transformation à notre RDD, ce qui a créé un nouveau RDD… mais nous ne pouvions pas l’utiliser, car il n’avait pas été stocké dans une variable!

In [12]:
# Les RDDs sont immuables! Notre transformation avait créé un autre RDD qui n'avait pas de 'nom' dans le Driver!

my_second_rdd = my_first_rdd.

my_second_rdd.

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

En créant des nouveaux RDDs à chaque transformation, Spark nous offre de la tolérance aux pannes! Spark enregistre chaque transformation dans un GAD. Si jamais il y a une panne dans un nœud au complet, ou dans un seul Executor, Spark peut immédiatement recalculer nos RDDs et nous ne perdons pas notre travail!

In [11]:
# Spark enregistre une lignée de nos transformations et peut les recalculer à tout moment en cas de panne!

my_second_rdd.

b'(40) PythonRDD[9] at RDD at PythonRDD.scala:48 []\n |   ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:480 []'

Mais attendons une minute… Spark produit des nouveaux RDDs à chaque Transformation, et nous avons vu que Spark garde nos données en mémoire… Est-ce que cela veut dire que nous remplirons la mémoire assez vite en appliquant des Transformations successivement? 

La réponse est NON! Spark fait de  ‘l’évaluation paresseuse’ (aussi appelée ‘appel par nécessité'). En d’autres mots, Spark enregistre tous nos transformations sur un GAD sans réellement faire aucun calcul et sans charger quoi que ce soit en mémoire avant qu’une **Action** ne soit appliqué à un RDD!

Regardons ce concept en action de manière intuitive, en appliquant une longue chaîne de Transformations à un RDD et en comptant le temps que cela va prendre... 

In [5]:
# Spark fait de l'evaluation paresseuse: Aucun calcul n'est fait avant qu'une Action ne soit appliqué à un RDD:

%time my_third_rdd = 

CPU times: user 38 µs, sys: 0 ns, total: 38 µs
Wall time: 42.2 µs


... L'output nous indique une opération pratiquement instantanée! Ajoutons une Action à la chaîne et mesurons le temps d'exécution:

In [11]:
#La méthode reduce() est une Action. Pour une liste complète des Actions sur Spark lisez: https://spark.apache.org/docs/latest/rdd-programming-guide.html#actions 

%time my_third_rdd.

CPU times: user 8.99 ms, sys: 1.99 ms, total: 11 ms
Wall time: 919 ms


40

Et voici des bonnes nouvelles pour ceux qui n'aiment pas la syntaxe ‘lambda function. Ceci marche aussi bien:’

In [12]:
%time my_third_rdd.

CPU times: user 8.28 ms, sys: 445 µs, total: 8.72 ms
Wall time: 255 ms


40

Les RDDs sont un concept très puissant et si vous devez vous rappeler d’une seule chose après l’atelier, ça devrait être ceci: RDDs sont une manière simple de faire du **Parallélisme de Donnée**.

En d’autres mots, vous pouvez écrire votre code presque exactement comme vous le feriez dans un programme serial (càd, pas parallèle) et Spark exécutera votre code sur des morceaux de votre jeu de données tous en même temps.

Tout ce que vous devez faire c’est d’envelopper votre code habituel avec un, ou plusieurs, des méthodes de l’API RDD et savoir ce qu’il y a dans les éléments des Partitions, ce qui vous permettra de choisir les bonnes méthodes à utiliser. Une fois que vous l’avez fait, Spark s’occupera de la partie 'Parallélisme de Donnée’!

Voici un exemple un peu plus complexe: utilisons Spark pour multiplier chaque élément d’un array numpy par un chiffre choisi aléatoirement!

Pourquoi est-ce plus complexe? Parce que nous ferons du Parallélisme de Donnée non pas sur un objet Python natif (comme une liste), mais un objet créé par une bibliothèque non-native: numpy. 

Commençons par créer l’objet: un array 1-d de 100 éléments.

In [13]:
import numpy as np

an_object = np.linspace(0,1,100)

ModuleNotFoundError: No module named 'numpy'

In [ ]:
an_object

In [ ]:
my_new_rdd = sc.

Vous serez peut-être tentés à faire comme dans les examples précedents et faire ce que vous auriez fait normalement sans Spark:

In [ ]:
my_new_rdd.

Cela ne marchera pas et vous aurez une erreur si vous roulez ce code sur un Cluster (et non pas sur un seul ordinateur). Pourquoi? Parce que nous avons importé la biliothèque <code>numpy</code> sur le Driver, mais nous voulons que les Executors puissent l'utiliser... nous devons dire aux Executors qu'ils doivent importer numpy aussi!

In [ ]:
def multiply_by_random(x):
   

In [ ]:
my_new_rdd.

Ça devrait avoir marché! Mais est-ce la meilleure façon de le faire? Rappelez-vous: la méthode <code>map</code> applique n'importe quoi que vous lui donnez comme argument à chaque élément de chaque partition!

Est-ce que cela veut dire que nous avons importé <code>numpy</code> 100 fois dans notre example? 

Oui!

Voilà une bonne opportunité pour parler d'une autre Transformation très utile: 

In [ ]:
def partition_multiply_by_random(x):
   

In [ ]:
my_new_rdd.

La méthode <code>mapPartition</code> applique ce que vous lui donnez comme argument à chaque **Partition**, mais avec une différence importante: votre fonction doit itérer sur les éléments de la Partition. Donc, en pratique, cette méthode appliquera, elle aussi, votre fonction aux éléments s'une Partition, mais vous aurez plus de fléxibilité pour faire des choses comme importer une biliothèque seulement une fois par Partition... ou n'importe quoi d'autre qui ne doit pas être executé à repetition pour chaque élément d'une Partition. 

**Avis important:** si votre code importe des bibliothèques, vous devez vous certifier qu'elles sont installés dans tous les noeuds du Cluster! En général cela veut dire que vous devez demander à votre administrateur de le faire pour vous...

Nous verons d'autres options pour passer les dépandances de votre code au Cluster dans la 2eme journée d'atelier!

## Example guidé  1 - Analyse des logs du site web de la NASA

Jusqu’à présent nous avons vu des exemples simples pour montrer le fonctionnement de l’API RDD et de quelques de ses Transformations et Actions. Examinons maintenant un exemple plus proche de la réalité: prenons un fichier ‘semi-structuré’ relativement gros et transformons-le en quelque chose qu’un Data Scientist serait prêt à utiliser. Tant qu’à le faire, nous allons nous servir de Spark pour  un peu de Data Science avec ces données en même temps.

Ce fichier est un log standard d’un webserver Apache. Dedans, nous trouverons un mois au complèt de logs d’accès au site de la NASA dans l’année lointaine de 1995.

Le log contient l’information suivante:

1. L’adresse IP ou le nom DNS à l’origine de l’accèst
2. L’horodatage de l’accès en format "dd/Mon/YYYY:hh:mm:ss Timezone"
3.Le type de requête (méthode HTTP) et l’adresse de la ressource demandée, ainsi que la version du protocole utilisé. 
4.Le code de statut retourné par le serveur (200 OK, 404 Not Found etc...)
5. La taille de la ressource.

Nous utiliserons la méthode <code>textFile</code> pour charger le fichier. Cette méthode, tout comme la méthode <code>parallelize</code>, transforme les données dans le fichier en RDD. Il y a deux choses importantes à savoir à propos de cette méthode:

Dans un vrai cluster Spark, l’endroit où le fichier est stocké (l’argument que nous allons passer à la méthode <code>textFile</code>) doit être visible et accessible à tous les nœuds du cluster. Très souvent, cet endroit sera un path dans un Hadoop Distributed File System (HDFS), mais ça pourrait également être n’importe quel Système d’Archives sur Réseau, un path monté sur tous les noeuds, un bucket sur Amazon S3… tant que l’endroit soit visible et accessible sur tous les noeuds! 
Cette méthode transforme **chaque ligne** du fichier en un élément d’une Partition. Donc, **peu importe le format du fichier**, quand il est transformé en RDD, **chaque ligne** (démarquée par un ‘\n’) devient un élément d’une Partition.

Sans plus attendre, passons à l'example!

In [3]:
nasa_logs = sc.textFile('data/NASA_access_log_Jul95')

Un bon prémier pas dans n'importe quel problème d'analyse de données c'est de les regarder pour avoir une idée de la nature du problème. L'API RDD contient l'Action <code>take</code>, qui nous permet de transferer un nombre limité d'éléments (rappelez-vous: un élément dans cet exemple est une ligne du fichier original) du Cluster au Driver où nous pouvons les voir. Il est important de ne pas transferer trop d'éléments au Driver, car vous pourrez dépasser sa capacité en mémoire!

In [20]:
nasa_logs.take(5)

['199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245',
 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085',
 'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0',
 '199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179']

Une autre bonne pratique c'est de compter le nombre total d'éléments pour avoir une idée de la taille du problème. Nous pouvons nous servir de la méthode <code>count</code> pour le faire:

In [ ]:
nasa_logs.count()

Maintenant que nous avons vu de quoi nos données ont l'air, séparer non données en morceaux délimités par un ' ' (espace) semble un bon premier pas:

In [4]:
nasa_logs.

[['199.72.81.55',
  '-',
  '-',
  '[01/Jul/1995:00:00:01',
  '-0400]',
  '"GET',
  '/history/apollo/',
  'HTTP/1.0"',
  '200',
  '6245'],
 ['unicomp6.unicomp.net',
  '-',
  '-',
  '[01/Jul/1995:00:00:06',
  '-0400]',
  '"GET',
  '/shuttle/countdown/',
  'HTTP/1.0"',
  '200',
  '3985'],
 ['199.120.110.21',
  '-',
  '-',
  '[01/Jul/1995:00:00:09',
  '-0400]',
  '"GET',
  '/shuttle/missions/sts-73/mission-sts-73.html',
  'HTTP/1.0"',
  '200',
  '4085'],
 ['burger.letters.com',
  '-',
  '-',
  '[01/Jul/1995:00:00:11',
  '-0400]',
  '"GET',
  '/shuttle/countdown/liftoff.html',
  'HTTP/1.0"',
  '304',
  '0'],
 ['199.120.110.21',
  '-',
  '-',
  '[01/Jul/1995:00:00:11',
  '-0400]',
  '"GET',
  '/shuttle/missions/sts-73/sts-73-patch-small.gif',
  'HTTP/1.0"',
  '200',
  '4179']]

Ensuite, disons que nous ne sommes pas intéressées par les lignes où il y a des données manquantes. En d'autres mots, nous voulons garder seulement les lignes où tous les 10 éléments sont présents. Nous nous servirons de la méthode <code>filter</code> pour filtrer tous les lignes où nous n'avons pas tous les 10 éléments:

In [9]:
nasa_logs.

Nous disons que les logs d'un webserver sont 'semi-structurés' pour une raison: nous pouvons être pas mal certains que chaque ligne aura le même format. Cela veut dire que tous les élément dans nos Partitions se ressembleront  entre eux après notre premier pas. Nous pouvons aussi être certains que les mêmes caractères dont nous n'avons pas besoin se trouveront dans les éléments de toutes les Partitions de notre RDD. Le prochain pas sera de les éliminer:

In [4]:
nasa_logs_structured = 

Vous vous demandez peut-être si utiliser la méthode <code>take</code> constantment pour examiner nos résultats est une bonne idée... et la réponse est non. À chaque fois que nous l'utilisons, Spark crée doit 'materializer' un nouveau RDD et, pour ce faire, le CLuster doit travailler. Dans la vraie vie, vous aurez rarement un Cluster Spark entièrement pour vous. Vous devez alors essayer de minimizer le nombre de fois où vous demandez au Cluster de travailler et, par consequence, vous devez minimizer le nombre de fois où vous demandez au Cluster de passer des données au Driver.

Donc, en pratique, un idée serait d'utiliser la méthode <code>sample</code> pour extraire un échantillon de vos données et les examiner dans le Driver. Une fois que vous aurez une bonne idée de ce que votre code devra faire avec ces données, vous pourrez passer au Cluster. La méthode <code>take</code> fonctionnerait bien aussi, mais utiliser un échantillon aléatoire au lieu des premiers N éléments de votre RDD est presque toujours une meilleure idée.

In [ ]:
# Ici nous prenons 0.001% du RDD en échantillon. Dans un jeu de Megadonnées, même cette petite proportion peut signifier une quantité massive de données que votre Driver ne pourra pas charger!

local_sample = nasa_logs.sample(withReplacement=False,fraction=0.0001).collect()

print(local_sample)

Notre RDD devrait maintenant contenir les éléments suivants: IP/NAME_OF_ORIGIN, DATE/TIME, TIMEZONE, REQUEST_METHOD, RESOURCE_REQUESTED, PROTOCOL, STATUS_CODE, SIZE_OF_RESOURCE

Ça resemble pas mal à un fichier CSV, le format préféré des Data Scientists!

Nous pouvons donc sauvegarder nos données dans un endroit où votre équipe de Data Scientists pourra aller les chercher.

Malheureusement, l'API RDD n'a pas de méthode pour écrire des fichiers CSV directement: nous devrons ajouter les virgules et forcer notre RDD à devenir un CSV avant de le sauvegarder:

In [11]:
def CSVfy(rdd_element):
  

nasa_logs_structured.map(CSVfy).take(5)

['199.72.81.55,-,-,01/Jul/1995:00:00:01,-0400,GET,/history/apollo/,HTTP/1.0,200,6245',
 'unicomp6.unicomp.net,-,-,01/Jul/1995:00:00:06,-0400,GET,/shuttle/countdown/,HTTP/1.0,200,3985',
 '199.120.110.21,-,-,01/Jul/1995:00:00:09,-0400,GET,/shuttle/missions/sts-73/mission-sts-73.html,HTTP/1.0,200,4085',
 'burger.letters.com,-,-,01/Jul/1995:00:00:11,-0400,GET,/shuttle/countdown/liftoff.html,HTTP/1.0,304,0',
 '199.120.110.21,-,-,01/Jul/1995:00:00:11,-0400,GET,/shuttle/missions/sts-73/sts-73-patch-small.gif,HTTP/1.0,200,4179']

In [ ]:
csv_to_be_saved = nasa_logs_structured.map(CSVfy)

csv_to_be_saved.saveAsTextFile('nasa_logs.csv')

La méthode <code>saveAsTextFile</code> a les mêmes particularités que son cousin <code>textFile</code>: l'endroit ou vous souhaitez sauvegarder vos données doit être visible et accessible sur tous les noeuds du Cluster. Comme avant, cet endroit sera tipiquement un path sur un Hadoop DFS.

Si vous ne souhaitez pas sauvegarder vos données dans un Système d'Archives Distribué comme HDFS, vous pouvez toujours utiliser la méthode <code>collect</code> pour apporter la totalité de votre RDD au Driver, où vous pouvez sauvegarder les données dans votre disque local en utilisant vos fonctions ou bibliothèques préférées sans vous soucier avec Spark. Mais encore là, le point d'avoir un cluster Spark est de pouvoir travailler avec des quantités massives de données qui ne rentreraient pas nécessairement dans votre disque local.

Vous vous demandez peut-être 'comment ça se fait que Spark n'a pas de méthode to_csv() comme padas pour écrire des CSV directement?', en remarquant que notre solution ne fonctionerait surement pas s'il y avait des virgules **à l'interieur des éléments** de notre RDD.

Vous auriez raison.

Il se trouve que Spark **a** en fait une méthode pour écrire des CSVs. Cette méthode est capable de gérer des  caractères speciaux, guillemets, virgules et tous les autres problèmes communs quand il est question de travailler avec des CSVs. Cette méthode n'est pas dans API RDD toutefois, mais dans l'API SparkSQL, dont nous parlerons à la 2eme journée de l'atelier.

Mais ça suffit de parler de CSVs pour le moment! Profitons de notre RDD fraichement structuré pour voir si nous serions capables faire un peu de Data Science directement sur Spark avec l'API RDD! Trouvons d'où est venu le plus grand nombre d'accèes au site de la NASA dans note jeu de données.

Pour le faire, rappelons-nous de la discussion sur la plateforme Hadoop et servons-nous d'un peu de Map-Reduce:

In [12]:
# Prenons chaque ligne de notre log structuré et créons une pair clé-valeur

nasa_logs_structured.

[('199.72.81.55', 1),
 ('unicomp6.unicomp.net', 1),
 ('199.120.110.21', 1),
 ('burger.letters.com', 1),
 ('199.120.110.21', 1)]

In [6]:
# Contrairement à reduce(), reduceByKey() n'est pas une Action!

nasa_logs_structured.

[(17572, 'piweba3y.prodigy.com'),
 (11591, 'piweba4y.prodigy.com'),
 (9868, 'piweba1y.prodigy.com'),
 (7852, 'alyssa.prodigy.com'),
 (7573, 'siltb10.orl.mmc.com')]

## Exercise 1 - à quelle date la NASA a eu le plus de traffic?

C'est votre tour! Prenez notre RDD <code>nasa_logs_structured</code> et trouvez l'horodatage du moment où le web serveur a enregistré le plus haut montant de données servis. Si vous voulez un défi, essayez de trouver le **Jour** où le serveur a servi le plus de données!



HINT: Quelques requêtes ne routernent aucune donnée, la taille de la ressource dans ces cas est "-".

HINT2: Tous les éléments dans notre RDD sont des Strings... 

In [22]:
nasa_logs_structured.persist()

PythonRDD[48] at RDD at PythonRDD.scala:48

In [24]:
nasa_logs_structured.is_cached

True

In [20]:
nasa_logs_structured.

[(6875029, '07/Jul/1995:14:03:32'),
 (3161433, '07/Jul/1995:10:28:56'),
 (3160666, '14/Jul/1995:09:11:29'),
 (3155499, '09/Jul/1995:09:22:14'),
 (3102848, '03/Jul/1995:12:30:07')]

## Exercise 2 - Quelle ressource a eu le plus de accès uniques?

Êtes-vous capables de trouver quel ressource de la NASA a eu le plus de visiteurs ou requêtes dans notre jeu des données?

HINT: La méthode <code>distinct</code> fais exactement ce que son nom suggère.


In [6]:
nasa_logs_structured.

[(49583, '/images/NASA-logosmall.gif'),
 (49049, '/images/KSC-logosmall.gif'),
 (29729, '/images/MOSAIC-logosmall.gif'),
 (29490, '/images/USA-logosmall.gif'),
 (29244, '/images/WORLD-logosmall.gif')]

## Exercise 3 - Word count

Si nous prenons l'élément de notre RDD où se trouvent les noms de ressources et que nous remplaçons les "/"s et les "."s par des " "s, nous avons des 'mots'. Combien de mots avons-nous dans le jeu de données et quel est le mot le plus fréquent?

HINT: Le code du programe word count est sur nos slides!
HINT2: Utilisez la méthode <code>count</code>.

In [11]:
words = nasa_logs_structured.

In [12]:
words.

[('', 2024051),
 ('countdown', 184637),
 ('mission-sts-73', 2327),
 ('ksclogo-medium', 58615),
 ('facts', 8619)]

Example guidé 2 - Une nuit au musée

L'API RDD est très puissante, mais seule elle a des serieuses limites. Ironiquement, une des plus grosses limites est son utilité quand il est question de données structurées... comme des fichiers CSV. 

Nous avons eu un aperçu de ce problème avec l'example du site web de la NASA. Maintenant nous nous tournons vers un CSV plus representatif de la vraie vie pour l'illustrer et nous profiterons pour voir pour la première fois l'API SparkSQL en action.  

Le fichier ci-dessous contient des données sur toutes les pièces d'art du Metropolitan Museum of Art à New York. Tel que nous l'avons déjà vu, l'API RDD chargera des fichiers de n'importe quel format comme un fichier texte commun.

In [13]:
museum_data = sc.textFile('data/met_data/MetObjects.csv')

In [14]:
museum_data.take(5)

['Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constiuent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL',
 '1979.486.1,False,False,False,1,,The American Wing,1979,Coin,One-dollar Liberty Head Coin,,,,,,16429,Maker,,James Barton Longacre,"American, Delaware County, Pennsylvania 1794–1869 Philadelphia, Pennsylvania",,"Longacre, James Barton",American,1794      ,1869      ,,http://vocab.getty.edu/page/ulan/500011409,,1853

In [15]:
museum_data.count()

587818

In [ ]:
museum_data_split = museum_data.map(lambda line : line.split(","))

In [ ]:
museum_data_split.take(1)

In [ ]:
from pyspark.sql import SQLContext

In [ ]:
sqlContext = SQLContext(sc)

In [ ]:
museum_dataframe = sqlContext.read.options(header='true').csv('data/met_data/MetObjects.csv')

In [ ]:
museum_dataframe

In [ ]:
museum_dataframe.head(1)